In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import os
import optuna

/home/echavemann/.cache/pypoetry/virtualenvs/kaggle-C0gLnRWh-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
gender_submission = pd.read_csv("gender_submission.csv")
data = pd.concat([train, test], sort=False)
data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True) #NA = S
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int) #Categorical Embarked
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'] = data['Age'].fillna(data.groupby(['Sex', 'Pclass'])['Age'].transform('median'))
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,2
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,2
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,1


In [4]:
delC = ['Name', 'Cabin', 'Ticket', 'PassengerId']
data.drop(delC, axis = 1, inplace = True)
train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis = 1)

In [5]:
params = {
    'objective':'binary',
    'learning_rate': 0.01,
    'num_leaves': 4,
    'feature_fraction' : 0.9,
    'min_data_in_leaf': 35,
    'max_depth': 2,
    'max_bin': 25
}
num_round = 5000

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.3, random_state=42, stratify = y_train)
cats = ['Embarked', 'Pclass', 'Sex']

In [7]:
lgbTrain = lgb.Dataset(X_train, y_train, categorical_feature = cats, free_raw_data=False)
lgbEval = lgb.Dataset(X_valid, y_valid, reference=lgbTrain, categorical_feature=cats, free_raw_data=False)

In [8]:
def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbose': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': 1,
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth':trial.suggest_int('max_depth', 1, 50),
    }
    bst = lgb.train(param,
                    lgbTrain,
                    valid_sets = [lgbTrain, lgbEval],
                    num_boost_round = num_round,
                    callbacks=[lgb.early_stopping(50)]
                   )
    preds = bst.predict(X_valid, num_iteration = bst.best_iteration)
    y_pred_binary = [1 if pred > 0.5 else 0 for pred in preds]
    from sklearn.metrics import log_loss
    return log_loss(y_pred_binary, y_valid)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)
study.best_trial

[I 2024-01-10 18:01:04,522] A new study created in memory with name: no-name-905544d3-7641-42a9-9d30-d6428e9b712f


Training until validation scores don't improve for 50 rounds


[I 2024-01-10 18:01:05,487] Trial 0 finished with value: 8.472948371322316 and parameters: {'lambda_l1': 5.161475955982295e-05, 'lambda_l2': 7.992612250902994e-06, 'num_leaves': 245, 'bagging_fraction': 0.47715473887164084, 'bagging_freq': 4, 'min_child_samples': 69, 'max_depth': 39}. Best is trial 0 with value: 8.472948371322316.


Early stopping, best iteration is:
[99]	training's binary_logloss: 0.423492	valid_1's binary_logloss: 0.456892


[LightGBM] [Fatal] Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.
You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.
[LightGBM] [Fatal] Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.
You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.


...

[I 2024-01-10 18:11:02,414] Trial 198 finished with value: 6.859053443451398 and parameters: {'lambda_l1': 1.5304264295302004e-07, 'lambda_l2': 1.0047284364547196e-05, 'num_leaves': 65, 'bagging_fraction': 0.8229355131908245, 'bagging_freq': 6, 'min_child_samples': 55, 'max_depth': 8}. Best is trial 41 with value: 5.783123491537453.


Early stopping, best iteration is:
[35]	training's binary_logloss: 0.392587	valid_1's binary_logloss: 0.431508
Training until validation scores don't improve for 50 rounds


[I 2024-01-10 18:11:02,982] Trial 199 finished with value: 5.783123491537453 and parameters: {'lambda_l1': 2.2095669580000568e-08, 'lambda_l2': 0.40071048340533555, 'num_leaves': 245, 'bagging_fraction': 0.948018616816114, 'bagging_freq': 5, 'min_child_samples': 61, 'max_depth': 37}. Best is trial 41 with value: 5.783123491537453.


Early stopping, best iteration is:
[53]	training's binary_logloss: 0.366202	valid_1's binary_logloss: 0.424511


FrozenTrial(number=41, state=1, values=[5.783123491537453], datetime_start=datetime.datetime(2024, 1, 10, 18, 3, 25, 717503), datetime_complete=datetime.datetime(2024, 1, 10, 18, 3, 26, 378111), params={'lambda_l1': 0.6409449461475386, 'lambda_l2': 2.571060917000477e-06, 'num_leaves': 47, 'bagging_fraction': 0.9415328816176557, 'bagging_freq': 6, 'min_child_samples': 66, 'max_depth': 9}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'bagging_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_freq': IntDistribution(high=7, log=False, low=1, step=1), 'min_child_samples': IntDistribution(high=100, log=False, low=5, step=1), 'max_depth': IntDistribution(high=50, log=False, low=1, step=1)}, trial_id=41, value=None)

In [8]:
best = study.best_params
best['objective'] = 'binary'

In [9]:
bst = lgb.train(best, 
                lgbTrain,
                valid_sets = [lgbTrain, lgbEval],
                verbose_eval=10,
                num_boost_round=num_round,
                callbacks=[lgb.early_stopping(50)])

[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000714 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 205
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


In [10]:
preds = bst.predict(X_test, num_iteration = bst.best_iteration)
preds = (preds > 0.5).astype(int)

In [11]:
submission = gender_submission
submission['Survived'] = preds
submission.to_csv("submission.csv", index=False)
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
